<div style="text-align:left;">
  <a href="https://code213.tech/" target="_blank">
    <img src="../images/code213.PNG" alt="QWorld">
  </a>
  <p><em>prepared by Latreche Sara</em></p>
</div>


###  Flavors of Boosting

In this notebook, we build a **photometric redshift estimator** using several **boosting methods**, including:

- **AdaBoost**
- **Gradient Boosted Trees (GBM)**
- **Histogram-Based Gradient Boosting (HistGBM)**
- **XGBoost**

We also explore how to improve model performance using **RandomizedSearchCV** for hyperparameter tuning.

**Objective**:  
Estimate **photometric redshifts** from observations of galaxy magnitudes in six photometric bands:  
**u, g, r, i, z, y**.
---

 **Reference Paper**:  
We aim to reproduce and improve upon the results of this paper:  
[Zou et al., 2019 — arXiv:1903.08174](https://arxiv.org/abs/1903.08174)




In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14)
matplotlib.rc('ytick', labelsize=14)
#matplotlib.rcParams.update({'figure.autolayout': True})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, GridSearchCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor

### We can read the data set with the selections applied in the previous notebook.

In [ ]:
sel_features = pd.read_csv('sel_features.csv', sep = '\t')

In [ ]:
sel_target = pd.read_csv('sel_target.csv')

In [ ]:
sel_features.shape

In [ ]:
sel_target.values.ravel() #changes shape to 1d row-like array

#### In the notebook "BoostingDecisions", we showed that for AdaBoost, stacking learners that are too weak doesn't help.

This allows us to run a more informed parameter optimization.


In [ ]:
# Step 1: Define the parameter grid for the AdaBoostRegressor.
# Each key corresponds to a parameter, and its values are the options to try.

parameters = {
    # The maximum depth of each decision tree (base learner).
    # Shallow trees (e.g., 6) are weaker learners; deeper trees (10, None) are stronger.

    # Type of loss function to minimize when updating weights:
    # 'linear' treats all errors equally, 'square' penalizes larger errors more.

    # Number of boosting stages (iterations).
    # More estimators = potentially better performance but longer training.
    'n_estimators': [20, 50, 100],

    # Learning rate shrinks the contribution of each tree.
    # Smaller values (like 0.3) mean slower but often more robust learning.
    # Larger values (like 1.0) can speed up learning but may overfit.
}

# Calculate total number of model combinations: 3 depths × 2 losses × 3 n_estimators × 3 learning_rates = 54 models

# Step 2: Instantiate the GridSearchCV object with 5-fold cross-validation
model =
    ,  # Boosting model using decision trees
      # Grid of hyperparameters to search
      # 5-fold CV with shuffling
      # Print progress
     # Use 4 CPU cores for parallel processing
   # Store training scores as well
)

# Step 3: Fit the model to the selected features and target

# Step 4: Print the best score and the combination of parameters that gave it
      # Format R^2 score


We can take a look at the winning model scores; in this case, we also pay attention to the standard deviation of test scores, because we want to know what differences are statistically significant when we compare different models.

In [ ]:
### Grouped Grid Search Results


#### 1. `base_estimator__max_depth`
#This controls how deep each decision tree is:
#- Smaller depths (e.g., 6) limit model complexity (weaker learners).
#- Larger depths (e.g., None = unlimited) allow more complex models.

#### 2. `learning_rate`
#Controls how much each tree contributes to the overall prediction:
#- Smaller values like `0.3` lead to slower learning but potentially better generalization.
#- Larger values like `1.0` can speed up training but risk overshooting the optimal solution.

#### 3. `n_estimators`
#The number of boosting rounds (trees):
#- Too few may underfit.
#- Too many can overfit unless regularized (via `learning_rate`).

#### 4. `loss`
#Specifies the loss function for regression:
#- `'linear'`: emphasizes absolute differences.
#- `'square'`: penalizes large errors more (sensitive to outliers).

#By grouping and averaging scores, we gain insights into which parameter values are most effective overall.


We can see that the standard deviation is 0.03 - giving us a hint of what's significant - and that a few different models have similar scores. If you change the random seed in the cross validation, the scores will change by a similar amount, and the best model may change as well.

Additionally, the resulting scores will not be exactly reproducible because there is another random component in the adaptive learning set (this means that if you run the cross_validate function using the best model from above, you might get a different average score!)

Let's pick the best model and check the scores. We should do nested cross validation to get the generalization errors right - but if we are just comparing models, this is ok.

In [ ]:
bm = model.best_estimator_

In [ ]:
# Use cross_val_predict to get cross-validated predictions from the model `bm`
# sel_features: the input features
# sel_target: the target values, flattened using .values.ravel()
# cv: apply 5-fold cross-validation with shuffling and fixed random state
# Store the predicted values in y_pred_bm



Calculate outlier fraction and NMAD:
###  What is NMAD (Normalized Median Absolute Deviation)?

When evaluating regression performance—especially in astronomy and photometric redshift estimation—we often encounter **outliers** (extreme prediction errors) that can distort typical error metrics like RMSE or standard deviation.

To handle this, we use **NMAD**, a robust statistic that provides a more reliable measure of the **spread of errors**, even in the presence of outliers.

---

####  Why is NMAD important?

- **Robust to outliers**: Unlike standard deviation, NMAD is based on the **median**, not the mean, so a few large errors don’t overly influence the result.
- **Standardized**: It normalizes the error with respect to $z_{\text{spec}}$, the true redshift, allowing fair comparison across objects.
- **Recommended in astronomy**: Widely adopted in photometric redshift estimation literature (e.g. in Ilbert et al. 2006 and Sánchez et al. 2014).

---

#### NMAD Formula

$$
\text{NMAD} = 1.4826 \times \text{median} \left( \left| \frac{\Delta z - \text{median}(\Delta z)}{1 + z_{\text{spec}}} \right| \right)
$$

Where:

- $\Delta z = z_{\text{phot}} - z_{\text{spec}}$ is the prediction error
- $z_{\text{phot}}$ is the predicted redshift
- $z_{\text{spec}}$ is the true redshift
- The constant **1.4826** scales the result to match the standard deviation for a Gaussian distribution

---

####  In Practice

- Use NMAD to evaluate the **typical scatter** of your predictions.
- Use it **alongside outlier fraction** and **R²** for a complete view of model performance.


In [ ]:
# Compute the outlier fraction
# An outlier is defined as a prediction where the absolute error exceeds 0.15 × (1 + true value)


# Compute the NMAD (Normalized Median Absolute Deviation)
# Formula: NMAD = 1.48 × median(|Δz| / (1 + z_spec))



In [ ]:
print(np.round(len(np.where(np.abs(sel_target.values.ravel()-y_pred_bm)>0.15*(1+sel_target.values.ravel()))[0])/len(sel_target.values.ravel()),3))

print(np.round(1.48*np.median(np.abs(sel_target.values.ravel()-y_pred_bm)/(1 + sel_target.values.ravel())),3))

These are actually better than what we obtained for the Random Forests model! But is the difference statistically significant? One way to explore this is by generating several sets of predictions, and calculating their standard deviation.

In [ ]:
# Pick 8 different random seeds from the range [0, 100)
  # Used to vary the random state in cross-validation

# Initialize arrays to store Outlier Fraction (OLF) and NMAD for each seed


# Loop over each random seed
  # Track progress of the loop

    # Perform cross-validated predictions using the current seed


    # Compute outlier fraction: where absolute error > 0.15 × (1 + true redshift)


    # Compute NMAD: 1.48 × median of absolute normalized error


# Print the mean and standard deviation of OLF and NMAD across seeds
print('OLF avg/std:, {0:.5f}, {1:0.5f}'.format(olf.mean(), olf.std()))
print('NMAD avg/std:, {0:.5f}, {1:0.5f}'.format(NMAD.mean(), NMAD.std()))


In [ ]:
# Pick 8 different random seeds from the range [0, 100)
seeds = np.random.choice(100, 8, replace=False)  # Used to vary the random state in cross-validation

# Initialize arrays to store Outlier Fraction (OLF) and NMAD for each seed
olf = np.zeros(8)
NMAD = np.zeros(8)

# Loop over each random seed
for i in range(8):
    print('Iteration', i)  # Track progress of the loop

    # Perform cross-validated predictions using the current seed
    ypred = cross_val_predict(
        bm,
        sel_features,
        sel_target.values.ravel(),
        cv=KFold(n_splits=5, shuffle=True, random_state=seeds[i])
    )

    # Compute outlier fraction: where absolute error > 0.15 × (1 + true redshift)
    olf[i] = (
        len(np.where(np.abs(sel_target.values.ravel() - ypred) > 0.15 * (1 + sel_target.values.ravel()))[0])
        / len(sel_target.values.ravel())
    )

    # Compute NMAD: 1.48 × median of absolute normalized error
    NMAD[i] = (
        1.48 * np.median(np.abs(sel_target.values.ravel() - ypred) / (1 + sel_target.values.ravel()))
    )

# Print the mean and standard deviation of OLF and NMAD across seeds
print('OLF avg/std:, {0:.5f}, {1:0.5f}'.format(olf.mean(), olf.std()))
print('NMAD avg/std:, {0:.5f}, {1:0.5f}'.format(NMAD.mean(), NMAD.std()))


The result seems to be relatively solid, indicating that Boosting methods might be slightly better than RF when we take into account not just the R2 score, but the specific metrics we are monitoring for this problem.

### The next step is to compare Adaptive Boosting with different Gradient Boosted Trees algorithms.

We begin by using sklearn's GBM, then we move on to the lighter version, HistGBM, and finally we consider one of the most popular GBT-based algorithm, XGBoost.

We also take a look at the possibility of using a Randomized Search instead of a Grid Search in order to speed up our optimization process.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor


The parameters depend on the particular implementation.

In the sklearn formulation, the parameters of each tree are essentially the same we have for Random Forests; additionally we have the "learning_rate" parameter, which dictates how much each tree contribute to the final estimator, and the "subsample" parameters, which allows one to use a < 1.0 fraction of samples and introduce some regularization.

### We can run the optimization process for this algorithm on a similar grid to the one used for AdaBoost.

In [ ]:
# %%time
# This cell uses GridSearchCV to optimize hyperparameters of GradientBoostingRegressor.
# On a typical machine, it may take several minutes to run (~4.5 minutes here).

# Define the hyperparameter grid for the Gradient Boosting Regressor:
parameters = {
    'max_depth': [6, 10, None],                        # Depth of individual trees
    'loss': ['squared_error', 'absolute_error'],       # Loss function used for optimization
    'n_estimators': [20, 50, 100],                     # Number of boosting stages
    'learning_rate': [0.1, 0.3, 0.5]                   # Shrinks contribution of each tree
}

# Compute the total number of models that will be tested during GridSearch

# Instantiate the GridSearchCV object using 5-fold cross-validation
      # The model to tune
                   # The hyperparameter grid
      # Cross-validation scheme
                      # Verbosity level (prints progress)
                     # Number of parallel jobs (use -1 to use all CPUs)
          # Also store training scores

# Fit the GridSearchCV model to the selected features and target

# Print the best hyperparameters and the corresponding cross-validation score


These are comparable to what we obtained with AdaBoost (slightly lower, typically). We can check what happens to the outlier fraction and NMAD.

In [ ]:
# determie best model with method best_estimator
bm =

In [ ]:
y_pred_bm =

In [ ]:
print(np.round(len(np.where(np.abs(sel_target.values.ravel()-y_pred_bm)>0.15*(1+sel_target.values.ravel()))[0])/len(sel_target.values.ravel()),3))

print(np.round(1.48*np.median(np.abs(sel_target.values.ravel()-y_pred_bm)/(1 + sel_target.values.ravel())),3))


Overall, the performance of the two algorithms is similar, but one important difference is timing. To explore exactly the same parameter space, GBR took ~ 3 times longer than AdaBoost. Additionally, gradient boosted methods typically require more estimators, and we should explore more regularization parameters (e.g. subsampling) as well. In a nutshell, it would be great to speed up things.

### How can we make things faster?

We can improve on the time constraints in two ways: by switching to the histogram-based version of Gradient Boosting Regressor, and by using a Random Search instead of a Grid Search.

HistGradientBoostingRegressor (inspired by [LightGBM](https://lightgbm.readthedocs.io/en/latest/)) works by binning the features into integer-valued bins (the default value is 256, but this parameter can be adjusted; note however that 256 is the maximum!), which greatly reduces the number of splitting points to consider, and results in a vast reduction of computation time, especially for large data sets.

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
# Perform hyperparameter tuning for the HistGradientBoostingRegressor

# Define a grid of parameters:
# - max_depth: controls tree depth (6, 10, or unlimited)
# - loss: either squared error or absolute error
# - max_iter: number of boosting iterations (20, 50, or 100)
# - learning_rate: step size for updating predictions (0.1, 0.3, or 0.5)

# Calculate the total number of models that will be trained as the product
# of the number of options in each parameter list.

# Instantiate GridSearchCV using:
# - HistGradientBoostingRegressor as the estimator
# - the defined parameter grid
# - 5-fold cross-validation with shuffling and a fixed random state for reproducibility
# - verbosity for tracking progress
# - n_jobs=4 to parallelize across 4 cores
# - return_train_score=True to store training scores

# Fit the model to the selected training features and target values.

# Print out the best cross-validation score and the corresponding parameters.



In [ ]:
# Create a DataFrame from the cross-validation results stored in model.cv_results_
# This includes training and validation metrics for each combination of hyperparameters.

# Select and organize key columns for inspection:
# - 'params': the hyperparameter configuration used
# - 'mean_test_score': the average score on the validation folds
# - 'std_test_score': the standard deviation of the validation scores
# - 'mean_train_score': the average score on the training folds

# Sort the results in descending order of mean_test_score to find the best-performing configurations.

# Display the top entries in the sorted DataFrame to review the most promising models.


In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV.head()

Even for this relatively small data set, this is much faster (about 15x faster than GradientBoostingRegressor), giving us a chance to explore a wider parameter space (e.g. more trees, more options for learning rate). The trade-off is that we obtain a slight decrease in performance, compared with GBR. However, the standard deviation of test scores over the 5 CV folds suggests that this difference is not statistically significant.

Let's explore a wider parameter space here:


In [ ]:
#%%time
# This block times the execution of a GridSearchCV optimization using HistGradientBoostingRegressor.
# The process took approximately 2 minutes and 42 seconds.

# Define the hyperparameter grid:
# - 'max_depth': Controls tree depth; higher depth can capture more complexity but may overfit.
# - 'loss': Specifies the loss function used for training ('squared_error' for MSE, 'absolute_error' for MAE).
# - 'max_iter': Number of boosting iterations (trees); higher values improve performance but increase training time.
# - 'learning_rate': Shrinks the contribution of each tree; lower rates need more iterations.
# - 'early_stopping': Stops training when validation score stops improving; helps prevent overfitting.

# Calculate the total number of models that will be trained by multiplying the length of all parameter lists.

# Initialize the GridSearchCV object:
# - Use 5-fold cross-validation with shuffled splits.
# - Run parallel jobs using n_jobs=4 to speed up processing.
# - Set verbose=2 to display progress.

# Fit the model on the selected features and target values.

# Print out the best score and associated hyperparameters after the search completes.


In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV.head()

### Comparison with Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Finally, we can compare the performance and timings of the grid search above with the option of using a Randomized Search instead. We note that Random Search is usually preferable when we have a high-dimensional parameter space; its use is not particularly warranted here.

The number of iterations (the number of models that are considered) also needs to be adjusted, and depends on the dimensionality of the parameter space as well as the functional dependence of the loss function on the parameters. We will compare the timings with the cell above, where we explore 144 models, and only use 30 for the random search.

The references here explores various ways of running a parameter search.

Bergstra, J. and Bengio, Y., Random search for hyper-parameter optimization, The Journal of Machine Learning Research (2012)

Bergstra, James, et al. "Algorithms for hyper-parameter optimization." Advances in neural information processing systems 24 (2011).

In [ ]:
#%%time
# This block times the execution of a RandomizedSearchCV optimization using HistGradientBoostingRegressor.

# Define the hyperparameter space to explore:
# - 'max_depth': Tree depth options.
# - 'loss': Error function options.
# - 'max_iter': Number of boosting iterations.
# - 'learning_rate': How much each tree contributes to the final model.
# - 'early_stopping': Whether to stop early if no improvement.

# Compute the total number of possible parameter combinations (not all will be used).

# Set up RandomizedSearchCV:
# - Uses a random subset (n_iter=30) of the full parameter space.
# - Applies 5-fold cross-validation with shuffled data splits.
# - Uses 4 parallel jobs for faster computation.
# - Returns training scores to analyze overfitting.

# Fit the model to the selected features and target variable.

# Output the best parameters found and the corresponding cross-validation score.


The Randomized Search was able to find a comparably good solution in less than 1/5 of the time. As we mentioned, the true gains of a Randomized Search pertain to exploring high-dimensional spaces. It is also possible to use the Randomized Search to find the general area of optimal parameters, and then refine the search in that neighborhood with a finer Grid Search.

### Finally, we can compare with XGBoost.

[XGBoost](https://xgboost.readthedocs.io/en/latest/index.html#) stands for “Extreme Gradient Boosting”. It is sometimes known as "regularized" GBM, as it has a default regularization term on the weights of the ensemble, and is more robust to overfitting. It has more flexibility in defining weak learners, as well as the objective (loss) function (note that this doesn't apply to the base estimators, e.g. how splits in trees are chosen, but on the loss that is used to compute pseudoresiduals and gradients).

In [ ]:
import xgboost as xgb

Medium article explaining XGBoost: [here](https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7); some nice tutorials from XGBoost's site: [here](https://xgboost.readthedocs.io/en/latest/tutorials/index.html)

We can begin by using Grid Search and the original parameter space, in order to compare timings with GBM and HistGBM.

In [ ]:
# %%time

# This block times the execution of a GridSearchCV optimization using XGBoost (XGBRegressor).

# Define the hyperparameter grid to search:
# - 'max_depth': Maximum depth of each decision tree.
# - 'n_estimators': Number of boosting rounds (trees).
# - 'learning_rate': Step size shrinkage used to prevent overfitting.
# - 'objective': Specifies the learning task and corresponding loss function.

# Calculate total number of parameter combinations in the grid.

# Set up GridSearchCV with:
# - 5-fold cross-validation (KFold) and data shuffling.
# - Verbose output to monitor progress.
# - 4 parallel jobs for faster computation.
# - Training scores also returned for later evaluation.

# Fit the model using the selected input features and target.

# Print out the best parameters and corresponding score from the cross-validation process.


XGBoost is slightly more efficient than GBM, and achieves comparable results on a similar grid. We can use the Random Search to explore some more intensive models (more trees, lower learning rate), and add subsampling as an extra form of regularization.

In [ ]:
# %%time

# This block times the execution of a RandomizedSearchCV optimization using XGBoost (XGBRegressor).
# Approximate runtime: 3 minutes 36 seconds.

# Define the hyperparameter distributions to sample from:
# - 'max_depth': Maximum depth of trees.
# - 'n_estimators': Number of boosting rounds.
# - 'learning_rate': Step size shrinkage.
# - 'objective': Loss function used by the model.
# - 'subsample': Fraction of training samples used for each tree to prevent overfitting.

# Calculate total number of possible parameter combinations (for reference only).

# Set up RandomizedSearchCV with:
# - 5-fold cross-validation (KFold) and data shuffling.
# - Verbose output to track progress.
# - Parallel processing with 4 jobs.
# - Return training scores.
# - Limit the number of sampled combinations to 30 (`n_iter=30`).

# Fit the model on selected features and targets.

# Print the best hyperparameter combination and the associated cross-validation score.


In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV.head()

We are able to get slightly higher scores using this wider parameter space in combination with the Randomized Search, but again, the statistical significance of this increase is very low.

We can also look for outlier fraction and NMAD:

In [ ]:
y_pred_bm = cross_val_predict(bm, sel_features, sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle = True, random_state=5))

In [ ]:
print(len(np.where(np.abs(sel_target.values.ravel()-y_pred_bm)>0.15*(1+sel_target.values.ravel()))[0])/len(sel_target.values.ravel()))

print(1.48*np.median(np.abs(sel_target.values.ravel()-y_pred_bm)/(1 + sel_target.values.ravel())))

### Conclusion: all boosting algorithms behave fairly similarly for this data set. It might be worth simply using the fastest one (HistGBR + Random Search).